# Data Wrangling

* Merging Datasets together
* Pivoting data
* Grouping 


In [1]:
import pandas as pd
import numpy as np

## Merging Data

* Bringing disparate datasets together is one of the more powerful features of Pandas
* Like with Python lists, you can `append()` and `concat()` Pandas `Series` and `Dataframes`
* The `concat` is a module function, you call it directly from the pandas module (usually called `pd`)

In [2]:
# concatinate two series together
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2]) #note the Seres are passed as a list

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

In [3]:
# order matters
pd.concat([ser2, ser1])

4    D
5    E
6    F
1    A
2    B
3    C
dtype: object

In [4]:
# concatinate two dataframes
df1 = pd.DataFrame({"A":["A1", "A2"],
                    "B":["B1","B2"]},index=[1,2])
df2 = pd.DataFrame({"A":["A3", "A4"],
                    "B":["B3","B4"]},index=[3,4])
pd.concat([df1,df2])

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


* Pandas will automatically line up matching indexes

In [5]:
# concatinate dataframes horizontally
df1 = pd.DataFrame({"A":["A1", "A2"],
                    "B":["B1","B2"]},index=[1,2])
df2 = pd.DataFrame({"C":["C1", "C2"],
                    "D":["D1","D2"]},index=[1,2])
pd.concat([df1,df2], axis=1)

,A,B,C,D
1,A1,B1,C1,D1
2,A2,B2,C2,D2


* And pandas will gracefully handle mis-alignment

In [6]:
# What happens when indexes don't line up
df1 = pd.DataFrame({"A":["A1", "A2"],
                    "B":["B1","B2"]},index=[1,2])
df2 = pd.DataFrame({"A":["A3", "A4"],
                    "B":["B3","B4"]},index=[3,4])
pd.concat([df1,df2], axis=1)

,A,B,A,B
1,A1,B1,NaN,NaN
2,A2,B2,NaN,NaN
3,NaN,NaN,A3,B3
4,NaN,NaN,A4,B4


* The `append` function is a method of a Series/Dataframe and returns a new object

In [7]:
# append df2 to df1
df1.append(df2)

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


### Merging and Joining

* While `concat()` is useful it lacks the power to do complex data merging
* For example, I have two tables of different data but one shared column
* This is where the `merge()` function becomes useful because it lets you *join* datasets
* The concept of "join" has lots of theory and is a richly developed method for *joining* data

#### One-to-one joins

In [8]:
# create two dataframes with one shared column
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue', "Nancy"],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR', "Librarian"]})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})

In [9]:
# display df1
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
4,Nancy,Librarian


In [10]:
# display df2
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [11]:
# merge df1 and df2 into a new dataframe df3
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


* The new dataframe `df3` now has all of the data from df1 and df2
* The `merge` function automatically connected the two tables on the "employee" column
* But what happens when your data don't line up?

#### Many-to-one joins

* Sometimes there isn't a one to one relationshp between rows in  two datasets
* A *many-to-one* join lets you combine these datasets

In [12]:
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [13]:
# make another dataframe about the supervisor for each group
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
df4

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


In [14]:
# Merge df3 from above with the supervisor info in df4
pd.merge(df3,df4)

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


* Notice how the information about Guido, the manager for Engineering, is repeated.
* Pandas automatically fills in these values to maintain the tabular, 2 dimensional structure of the data
* While this might seem like duplicated data, it makes it easier to quickly look up Jake and Lisa's supervisor without consulting multiple tables

#### Many-to-many joins

* Let's combine the employee information with skills information
* Notice there isn't a one to one or even a one to many relationship between these tables
* Each group can have multiple skills, so **what do you think will happen?**

In [15]:
# Use the employee table specified above
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
4,Nancy,Librarian


In [16]:
# create a new dataframe with skills information
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR', 'Librarian'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization', 'nunchucks']})
df5

,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets
5,HR,organization
6,Librarian,nunchucks


In [17]:
pd.merge(df1, df5)

,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Engineering,coding
5,Lisa,Engineering,linux
6,Sue,HR,spreadsheets
7,Sue,HR,organization
8,Nancy,Librarian,nunchucks


* Amazing, Pandas merge capabilities are very useful when column names match up
* But what do you do if the names of your columns don't match?
* You could change column names...
* But that is crazy! Just use the `left_on` and `right_on` parameters to the `merge()` function

In [18]:
# Use the employee table specified above
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
4,Nancy,Librarian


In [19]:
df2 = df2.rename({"employee":"name"}, axis="columns")

In [ ]:
# lets try and merge them without specifying what to merge on
pd.merge(df1, df2)

* Gak, error! Pandas can't figure out how to combine them
* What are the column names I should specify?

In [21]:
# Now lets specify the column name 
pd.merge(df1, df2, left_on="employee", right_on="name" )

,employee,group,name,hire_date
0,Bob,Accounting,Bob,2008
1,Jake,Engineering,Jake,2012
2,Lisa,Engineering,Lisa,2004
3,Sue,HR,Sue,2014


* Notice we now have a redundant employee/name column, this is a by-product of merging different columns
* If you want to get rid of it you can use the `drop` method

In [22]:
# drop the name column, axis=1 means axis='col', which is confusing
pd.merge(df1, df2, left_on="employee", right_on="name" ).drop("name", axis=1)

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


* This is just a taste of merging and joining data
* We will cover more of this in the SQL and Relational Databases sessions

## Pivoting Data

* Sometimes you get what is called "long" or "stacked" data (streaming values from an instrument or periodic observational data)
* Data in this shape can be difficult to analyze

In [23]:
# load the CSV file
data = pd.read_csv("../4 - data management one/community-center-attendance.csv",
                  index_col="_id")
                   
# look at the first ten rows of the data
data.head(10)

,date,center_name,attendance_count
_id,,,
1,2018-06-08,Ormsby Community Center,10
2,2018-06-08,Paulson Community Center,19
3,2018-06-08,Phillips Community Center,107
4,2018-06-08,Ammon Community Center,81
5,2018-06-08,Brookline Community Center,33
6,2018-06-08,Jefferson Community Center,29
7,2018-06-08,Warrington Community Center,15
8,2018-06-08,West Penn Community Center,54
9,2018-06-07,Phillips Community Center,77


In [24]:
# How many rows we got?
data.shape

(18367, 3)

* These data are looooooong
* Each row represents a community center in Pittsburgh reporting how many people visited the center
* Given this shape it is possible to do some calculations, but it might make more sense to *pivot* the data so that each column is a community center and each row is a day

In [25]:
# Use the pivot function to make column values into columns
pivoted_data = data.pivot_table(index="date", # these values will be rows
                          columns="center_name", # these values will be columns
                          values="attendance_count" # these values will populate the table
                         )
pivoted_data.head()

center_name,Ammon / Josh Gibson Field,Ammon Community Center,Ammon Pool,Arlington Community Center,Arlington Field (Playground),Brookline Community Center,Dan Marino Field (Playground),Frick Environmental Center,Gladstone Field,Highland Pool,...,Paulson Community Center,Paulson Field,Phillips Community Center,Phillips Park Field,Schenley Ice Rink,Warrington Community Center,Warrington Field,West Penn Community Center,West Penn Fields,West Penn Pool
date,,,,,,,,,,,,,,,,,,,,,
2011-03-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN
2011-03-08,NaN,NaN,NaN,NaN,NaN,22.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
2011-03-09,NaN,NaN,NaN,NaN,NaN,60.0,NaN,NaN,NaN,NaN,...,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-03-10,NaN,1.0,NaN,NaN,NaN,54.0,NaN,NaN,NaN,NaN,...,NaN,NaN,32.0,NaN,NaN,2.0,NaN,1.0,NaN,NaN
2011-03-11,NaN,19.0,NaN,NaN,NaN,77.0,NaN,NaN,NaN,NaN,...,5.0,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* Now we can easily find out things about our favorite community centers

In [26]:
# Total number of people who have visited Magee
pivoted_data['Magee Community Center'].sum()

61377.0

In [27]:
# Average attendence per day at Magee
pivoted_data['Magee Community Center'].mean()

34.098333333333336

### Transpose

* Pandas has a handy function for *transposing* dataframes
* It just rotates the table making the columns rows and the rows columns

In [30]:
pivoted_data.T.head()

date,2011-03-07,2011-03-08,2011-03-09,2011-03-10,2011-03-11,2011-03-12,2011-03-13,2011-03-14,2011-03-15,2011-03-16,...,2018-05-29,2018-05-30,2018-05-31,2018-06-01,2018-06-02,2018-06-04,2018-06-05,2018-06-06,2018-06-07,2018-06-08
center_name,,,,,,,,,,,,,,,,,,,,,
Ammon / Josh Gibson Field,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ammon Community Center,NaN,NaN,NaN,1.0,19.0,1.0,NaN,NaN,48.0,4.0,...,87.0,46.0,96.0,30.0,NaN,62.0,122.0,125.0,133.0,81.0
Ammon Pool,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0,NaN,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arlington Community Center,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arlington Field (Playground),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* Now the Column and row indexes are swapped

## Grouping Data


* A common pattern in data analysis is splitting data by a key and then performing some math on all of the values with that key and finally combining it all back together
* This is commonly known in data circles as *split, apply, combine*


In [31]:
# create a dataframe to illustrate GroupBy
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6),
                   'counts':[45,234,6,2,1324,345], 
                   'things':['dog', 'cat', 'cat', 'dog', 'cat', 'cat']}
                 )
df

,key,data,counts,things
0,A,0,45,dog
1,B,1,234,cat
2,C,2,6,cat
3,A,3,2,dog
4,B,4,1324,cat
5,C,5,345,cat


In [32]:
# Dataframes have a method, groupby(), that takes a column name be be the grouping key
df.groupby('key')

In [33]:
for group in df.groupby('key'):
    print("Group for key:", group[0])
    print("Data:", group[1])
    print("Data Type:", type(group[1]))
    print()

Group for key: A
Data:   key  data  counts things
0   A     0      45    dog
3   A     3       2    dog
Data Type: <class 'pandas.core.frame.DataFrame'>

Group for key: B
Data:   key  data  counts things
1   B     1     234    cat
4   B     4    1324    cat
Data Type: <class 'pandas.core.frame.DataFrame'>

Group for key: C
Data:   key  data  counts things
2   C     2       6    cat
5   C     5     345    cat
Data Type: <class 'pandas.core.frame.DataFrame'>



* Cool, we can see that we have *split* our data into three groups
* Now, we need to tell Pandas what function to *apply* to each group
* We need to specify what kind of aggregation, transformation, or computation to perform on the group

In [34]:
# Tell pandas to add up all of the values for each key
df.groupby('key').sum()

,data,counts
key,,
A,3,47
B,5,1558
C,7,351


* Under the hood Pandas is creating a bunch of new Dataframes based on the grouping column values

In [35]:
# you can save the group object and run different aggregations
grouped_dataframe = df.groupby('key')
grouped_dataframe.sum()

,data,counts
key,,
A,3,47
B,5,1558
C,7,351


In [36]:
grouped_dataframe.mean()

,data,counts
key,,
A,1.5,23.5
B,2.5,779.0
C,3.5,175.5


* The following table summarizes some other built-in Pandas aggregations:

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``size()``               | Total number of items w/ NaNs   |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

* These are all methods of ``DataFrame`` and ``Series`` objects.

* You can also do multiple levels of grouping

In [37]:
df.groupby(['things','key']).count()

data  counts
things key              
cat    B       2       2
       C       2       2
dog    A       2       2

* What you are seeing is what is called a [Multilevel Index](https://pandas.pydata.org/pandas-docs/stable/advanced.html)
* Sadly, we don't have time to cover that topic, but this chapter on [Hierarchical Indexing](https://jakevdp.github.io/PythonDataScienceHandbook/03.05-hierarchical-indexing.html) in the [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/) is a great introduction to the topic.

## Split, Apply, Combine with real data

* Lets grab a dataset from the WPRDC, the [Allegheny County Jail Daily Census](https://data.wprdc.org/dataset/allegheny-county-jail-daily-census)

In [ ]:
# Grab three months of data
january17_jail_census = pd.read_csv("https://data.wprdc.org/datastore/dump/3b5d9c45-b5f4-4e05-9cf1-127642ad1d17")
feburary17_jail_census = pd.read_csv("https://data.wprdc.org/datastore/dump/cb8dc876-6285-43a8-9db3-90b84eedb46f")
march17_jail_census = pd.read_csv("https://data.wprdc.org/datastore/dump/68645668-3f89-4831-b1de-de1e77e52dd3")


In [ ]:
january17_jail_census.head()

* Use the concat function to combine all three into one dataframe
* Remember `concat` is a general pandas function so we call it with `pd.concat`
* It takes as an argument a list of things to combine

In [ ]:
# make a list of dataframes to combine
dataframes = [january17_jail_census, 
              feburary17_jail_census, 
              march17_jail_census]

# give the concat function the list
jail_census = pd.concat(dataframes)
jail_census

* Now we can do some calculations on the data
* Note, because this is a daily census it includes many of the same people, so this mean isn't statistically *meaningful* 
* We can still use these data for demonstration purposes 

In [ ]:
# Compute the average age ate booking by gender
jail_census.groupby('Gender')['Age at Booking'].mean()

In [ ]:
# compute the average age at booking by race then gender 
jail_census.groupby(['Race', 'Gender'])['Age at Booking'].mean()

If we look at the [data dictionary](https://data.wprdc.org/dataset/allegheny-county-jail-daily-census/resource/f0550174-16b0-4f6e-88dc-fa917e74b56c) we can see the following mapping for race categories
```
Race of Inmate
A-ASIAN OR PACIFIC ISLANDER
B-BLACK OR AFRICAN AMERICAN
H-HISPANIC 
I-AMERICAN INDIAN OR ALASKAN NATIVE
U-UNKNOWN
W-WHITE
```


In [ ]:
# Get the statistical summary of age at booking by gender
jail_census.groupby('Gender')['Age at Booking'].describe()

In [ ]:
# Compute the difference between Age at Booking and current age
age_difference = jail_census['Current Age'] - jail_census['Age at Booking']
age_difference.value_counts()

* This uses a mathematical funciton to compute the age difference for each row
* Then we use `value_counts` to count the age differences

In [ ]:
jail_census.groupby('Date').count()

In [ ]:
jail_census['year'] = jail_census['Date'].str.split("-").str[0]
jail_census['month'] = jail_census['Date'].str.split("-").str[1]
jail_census['day'] = jail_census['Date'].str.split("-").str[2]

jail_census.head()

In [ ]:
jail_census.groupby('month').count()

In [ ]:
jail_census.groupby('day').count()

* This is a really awkward way of dealing with time
* We shouldn't have to make a separate column for year, month, day
* There must be a better way to do this time stuff...
